In [29]:
"""
This cell is to get all the data for the ml model
"""

allX = []
allY = []

trainX = []
trainY = []
valiX = []
valiY = []
testX = []
testY = []

linesToUse = 1000000
totalLines = 0
# first try with just getting all data
with open('/users/cs/amaral/wikisim/wikification/learning-data/el-5000.txt', 'r') as f:
    for line in f:
        totalLines += 1
        if totalLines > linesToUse:
            break
        data = line.split(',')
        allX.append([float(data[2]), float(data[3]), float(data[4]), float(data[5]), float(data[6])])
        allY.append(int(data[1]))
        
# split 60, 20, 20
trainLines = int(totalLines * 0.75)
valiLines = int(totalLines * 0.0)
testLines = int(totalLines * 0.25)

for i in range(0, trainLines):
    trainX.append(allX[i])
    trainY.append(allY[i])

for i in range(trainLines, trainLines + valiLines):
    valiX.append(allX[i])
    valiY.append(allY[i])
    
for i in range(trainLines + valiLines, trainLines + valiLines + testLines):
    testX.append(allX[i])
    testY.append(allY[i])

In [30]:
""" This and other cells helped by: https://github.com/ogrisel/notebooks/blob/master/Learning%20to%20Rank.ipynb
This cell is to train the model.
"""

from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
etr.fit(trainX, trainY)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=5, min_weight_fraction_leaf=0.0,
          n_estimators=200, n_jobs=-1, oob_score=False, random_state=1,
          verbose=0, warm_start=False)

In [42]:
import numpy as np
printEval(etr, testX, testY, 0.45)
print
for i in np.arange(0.0, 1.0, 0.1):
    printEval(etr, testX, testY, i)
    print

BDB =  0.45 
TP: 5613 
FP: 574 
TN: 49949 
FN: 563

BDB =  0.0 
TP: 6134 
FP: 8991 
TN: 41532 
FN: 42

BDB =  0.1 
TP: 5993 
FP: 1909 
TN: 48614 
FN: 183

BDB =  0.2 
TP: 5897 
FP: 1204 
TN: 49319 
FN: 279

BDB =  0.3 
TP: 5788 
FP: 878 
TN: 49645 
FN: 388

BDB =  0.4 
TP: 5679 
FP: 667 
TN: 49856 
FN: 497

BDB =  0.5 
TP: 5533 
FP: 496 
TN: 50027 
FN: 643

BDB =  0.6 
TP: 5372 
FP: 374 
TN: 50149 
FN: 804

BDB =  0.7 
TP: 5177 
FP: 270 
TN: 50253 
FN: 999

BDB =  0.8 
TP: 4894 
FP: 174 
TN: 50349 
FN: 1282

BDB =  0.9 
TP: 4462 
FP: 91 
TN: 50432 
FN: 1714



In [40]:
def printEval(model, X, y, bdb = 0.5):
    predY = model.predict(X)
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for i in range(len(y)):
        if predY[i] > bdb and y[i] == 1:
            tp += 1
        elif predY[i] > bdb and y[i] == 0:
            fp += 1
        elif predY[i] <= bdb and y[i] == 1:
            fn += 1
        elif predY[i] <= bdb and y[i] == 0:
            tn += 1
            
    print 'BDB = ', bdb, '\nTP:', tp, '\nFP:', fp, '\nTN:', tn, '\nFN:', fn